In [29]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import random
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,-7.31,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,0.87,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,-2.44,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000


In [31]:
# ATA를 datetime 형태로 변환
train['ATA_datetime'] = pd.to_datetime(train['ATA'])

# 항구별 일별 도착 선박 수 계산
port_daily_arrivals = train.groupby(['ARI_PO', train['ATA_datetime'].dt.date]).size().reset_index(name='arrivals')

# 항구별 일별 평균 도착률 계산
port_daily_avg_arrival_rate = port_daily_arrivals.groupby('ARI_PO')['arrivals'].mean()

# 전체 데이터의 평균 대기 시간을 계산하여 일별 서비스률 추정
average_waiting_time = train['CI_HOUR'].mean()
daily_service_rate = 1 / average_waiting_time

# 데이터에 항구별 파생 변수 추가
train['port_daily_arrival_rate'] = train.apply(lambda row: port_daily_avg_arrival_rate[row['ARI_PO']], axis=1)
port_avg_waiting_time = train.groupby('ARI_PO')['CI_HOUR'].mean()
port_daily_service_rate = 1 / port_avg_waiting_time
train['port_daily_service_rate'] = train['ARI_PO'].map(port_daily_service_rate)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,ATA_datetime,port_daily_arrival_rate,port_daily_service_rate
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333,2020-10-15 04:03:00,8.710324,0.028614
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611,2019-09-17 02:55:00,7.542746,0.018029
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,14,65.30,66.39,56.94,603.193047,0.001743,98.827500,2019-02-23 06:43:00,9.475826,0.023371
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000,2020-09-18 22:06:00,2.185466,0.020537
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556,2022-08-13 12:57:00,1.527931,0.012597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000,2017-11-11 22:23:00,2.746103,0.010488
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000,2022-04-29 02:58:00,6.129946,0.051559
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500,2022-07-14 07:58:00,13.495088,0.009215
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000,2020-12-22 10:07:00,2.185466,0.020537


In [32]:
# 항구 및 요일별 일별 도착 선박 수 계산
port_dayofweek_daily_arrivals = train.groupby(['ARI_PO', train['ATA_datetime'].dt.dayofweek]).size().reset_index(name='arrivals')

# 항구 및 요일별 일별 평균 도착률 계산
port_dayofweek_avg_arrival_rate = port_dayofweek_daily_arrivals.groupby(['ARI_PO', 'ATA_datetime'])['arrivals'].mean()

# 항구 및 요일별 평균 대기 시간 계산
port_dayofweek_avg_waiting_time = train.groupby(['ARI_PO', train['ATA_datetime'].dt.dayofweek])['CI_HOUR'].mean()

# 항구 및 요일별 서비스률 추정 (가정: 하루에 처리 가능한 선박 수는 1/평균 대기 시간)
port_dayofweek_service_rate = 1 / port_dayofweek_avg_waiting_time

# 데이터에 항구 및 요일별 파생 변수 추가
train['port_dayofweek_arrival_rate'] = train.apply(lambda row: port_dayofweek_avg_arrival_rate[(row['ARI_PO'], row['ATA_datetime'].dayofweek)], axis=1)
train['port_dayofweek_service_rate'] = train.apply(lambda row: port_dayofweek_service_rate[(row['ARI_PO'], row['ATA_datetime'].dayofweek)], axis=1)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,ATA_datetime,port_daily_arrival_rate,port_daily_service_rate,port_dayofweek_arrival_rate,port_dayofweek_service_rate
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,43.16,40.96,1407.668330,0.001660,3.048333,2020-10-15 04:03:00,8.710324,0.028614,4807.0,0.039002
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,64.55,59.34,2089.046774,0.001614,17.138611,2019-09-17 02:55:00,7.542746,0.018029,3907.0,0.021508
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,66.39,56.94,603.193047,0.001743,98.827500,2019-02-23 06:43:00,9.475826,0.023371,1331.0,0.009466
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,43.15,41.11,1169.853455,0.000069,0.000000,2020-09-18 22:06:00,2.185466,0.020537,430.0,0.034673
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,93.65,88.11,1107.944894,0.000197,96.030556,2022-08-13 12:57:00,1.527931,0.012597,181.0,0.006117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,62.21,55.70,1333.609109,0.000360,65.850000,2017-11-11 22:23:00,2.746103,0.010488,764.0,0.007638
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,109.34,104.69,1955.103846,0.000552,0.000000,2022-04-29 02:58:00,6.129946,0.051559,3002.0,0.071653
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,99.10,95.78,1601.291086,0.002615,0.997500,2022-07-14 07:58:00,13.495088,0.009215,7030.0,0.010007
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,50.08,47.02,1191.353331,0.000069,0.000000,2020-12-22 10:07:00,2.185466,0.020537,573.0,0.026080


In [33]:
# 항구 및 요일별 rho (도착률/서비스률) 계산
port_dayofweek_rho = train['port_dayofweek_arrival_rate'] / train['port_dayofweek_service_rate']

# 시스템이 특정 인원(예: 5인 이상)일 확률
n = 5
P_n = (1 - port_dayofweek_rho) * (port_dayofweek_rho ** n)

# 시스템이 바쁜 확률
P_busy = port_dayofweek_rho

# 데이터에 파생 변수 추가
train['P_n'] = P_n
train['P_busy'] = P_busy

In [34]:
# 요일별 일별 도착 선박 수 계산
dayofweek_daily_arrivals = train.groupby(train['ATA_datetime'].dt.dayofweek).size().reset_index(name='arrivals')

# 요일별 일별 평균 도착률 계산
dayofweek_avg_arrival_rate = dayofweek_daily_arrivals.set_index('ATA_datetime')['arrivals'] / len(train['ATA_datetime'].dt.date.unique())

# 요일별 평균 대기 시간 계산
dayofweek_avg_waiting_time = train.groupby(train['ATA_datetime'].dt.dayofweek)['CI_HOUR'].mean()

# 요일별 서비스률 추정 (가정: 하루에 처리 가능한 선박 수는 1/평균 대기 시간)
dayofweek_service_rate = 1 / dayofweek_avg_waiting_time

# 요일별 rho (도착률/서비스률) 계산
dayofweek_rho = dayofweek_avg_arrival_rate / dayofweek_service_rate

# 시스템이 특정 인원(예: 5인 이상)일 확률
P_n_dayofweek = (1 - dayofweek_rho) * (dayofweek_rho ** n)

# 시스템이 바쁜 확률
P_busy_dayofweek = dayofweek_rho

# 데이터에 요일별 파생 변수 추가
train['P_n_dayofweek'] = train['ATA_datetime'].dt.dayofweek.map(P_n_dayofweek)
train['P_busy_dayofweek'] = train['ATA_datetime'].dt.dayofweek.map(P_busy_dayofweek)


# V3

In [35]:
train = train[train['DIST'] != 0]
train = train.reset_index(drop = True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,CI_HOUR,ATA_datetime,port_daily_arrival_rate,port_daily_service_rate,port_dayofweek_arrival_rate,port_dayofweek_service_rate,P_n,P_busy,P_n_dayofweek,P_busy_dayofweek
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.048333,2020-10-15 04:03:00,8.710324,0.028614,4807.0,0.039002,-3.505444e+30,123251.183611,-1.351226e+18,1051.615274
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,17.138611,2019-09-17 02:55:00,7.542746,0.018029,3907.0,0.021508,-3.592642e+31,181650.275277,-1.529832e+18,1073.597276
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,98.827500,2019-02-23 06:43:00,9.475826,0.023371,1331.0,0.009466,-7.727559e+30,140607.201668,-9.844177e+17,997.552656
3,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,96.030556,2022-08-13 12:57:00,1.527931,0.012597,181.0,0.006117,-6.711144e+26,29589.334999,-9.844177e+17,997.552656
4,TRAIN_000005,CN,NGG6,Container,81.435335,2015-09-08 14:24,Z156413,30.0,22,18100,...,42.078056,2015-09-08 14:24:00,9.475826,0.023371,5093.0,0.028268,-3.420287e+31,180167.930277,-1.529832e+18,1073.597276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220050,TRAIN_367433,IN,UJM2,Bulk,30.199074,2022-03-23 8:35,Y242521,30.0,2,63500,...,53.400833,2022-03-23 08:35:00,1.750577,0.015647,544.0,0.021243,-2.820130e+26,25608.316389,-1.412431e+18,1059.407138
220051,TRAIN_367434,CN,QQW1,Bulk,55.408765,2022-06-16 14:27,D236761,30.0,16,26500,...,83.960833,2022-06-16 14:27:00,2.826051,0.023430,1213.0,0.028873,-5.498079e+27,42011.696667,-1.351226e+18,1051.615274
220052,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,65.850000,2017-11-11 22:23:00,2.746103,0.010488,764.0,0.007638,-1.001397e+30,100023.444167,-9.844177e+17,997.552656
220053,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,0.997500,2022-07-14 07:58:00,13.495088,0.009215,7030.0,0.010007,-1.201726e+35,702480.641106,-1.351226e+18,1051.615274


In [36]:
'''
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])|
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)
'''

"\n# datetime 컬럼 처리\ntrain['ATA'] = pd.to_datetime(train['ATA'])|\ntest['ATA'] = pd.to_datetime(test['ATA'])\n\n# datetime을 여러 파생 변수로 변환\nfor df in [train, test]:\n    df['year'] = df['ATA'].dt.year\n    df['month'] = df['ATA'].dt.month\n    df['day'] = df['ATA'].dt.day\n    df['hour'] = df['ATA'].dt.hour\n    df['minute'] = df['ATA'].dt.minute\n    df['weekday'] = df['ATA'].dt.weekday\n\n# datetime 컬럼 제거\ntrain.drop(columns='ATA', inplace=True)\ntest.drop(columns='ATA', inplace=True)\n"

In [37]:
train.drop(['SAMPLE_ID'],axis=1,inplace=True)
test.drop(['SAMPLE_ID'],axis=1,inplace=True)

In [38]:
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])
train['year'] = train['ATA'].dt.year
train['month'] = train['ATA'].dt.month
train['day'] = train['ATA'].dt.day
train['weekday'] = train['ATA'].dt.dayofweek
train['rounded_hour'] = (train['ATA'].dt.hour + (train['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)
test['year'] = test['ATA'].dt.year
test['month'] = test['ATA'].dt.month
test['day'] = test['ATA'].dt.day
test['weekday'] = test['ATA'].dt.dayofweek
test['rounded_hour'] = (test['ATA'].dt.hour + (test['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)

# sin, cos 변환 함수 정의
def encode_cyclic_feature(data, column, max_val):
    data[column + '_sin'] = np.sin(2 * np.pi * data[column] / max_val)
    data[column + '_cos'] = np.cos(2 * np.pi * data[column] / max_val)
    return data

# 각 피처에 대해 sin, cos 변환 수행
train = encode_cyclic_feature(train, 'month', 12)
train = encode_cyclic_feature(train, 'day', 31)
train = encode_cyclic_feature(train, 'weekday', 7)
train = encode_cyclic_feature(train, 'rounded_hour', 24)
test = encode_cyclic_feature(test, 'month', 12)
test = encode_cyclic_feature(test, 'day', 31)
test = encode_cyclic_feature(test, 'weekday', 7)
test = encode_cyclic_feature(test, 'rounded_hour', 24)

train.drop(['ATA'],axis=1,inplace=True)
test.drop(['ATA'],axis=1,inplace=True)

test[['year', 'month', 'day', 'weekday', 'rounded_hour']].head()

,year,month,day,weekday,rounded_hour
0,2022,8,27,5,8
1,2022,3,27,6,21
2,2023,1,18,2,2
3,2016,8,2,1,1
4,2023,1,24,1,0


# V2

In [39]:
#train.drop(['ID'],axis=1,inplace=True)
#test.drop(['ID'],axis=1,inplace=True)
#train['ID'] = train['ID'].str[0]
#test['ID'] = test['ID'].str[0]


features_to_encode = ['ARI_CO', 'ARI_PO', 'year', 'month', 'day', 'weekday', 'rounded_hour']

for feature in features_to_encode:
    # mean encoding
    train[f'mean_enc_{feature}'] = train.groupby(feature)['CI_HOUR'].transform('mean')
    # std encoding
    train[f'std_enc_{feature}'] = train.groupby(feature)['CI_HOUR'].transform('std')

# 결과의 상위 몇 개 행을 출력하여 확인
mean_enc_maps = {feature: train.groupby(feature)['CI_HOUR'].mean().to_dict() for feature in features_to_encode}
std_enc_maps = {feature: train.groupby(feature)['CI_HOUR'].std().to_dict() for feature in features_to_encode}

for feature in features_to_encode:
    test[f'mean_enc_{feature}'] = test[feature].map(mean_enc_maps[feature])
    test[f'std_enc_{feature}'] = test[feature].map(std_enc_maps[feature])







# ARI_PO, weekday, rounded_hour 조합에 따른 mean 및 std encoding
'''
grouped = train.groupby(['ARI_CO', 'weekday', 'rounded_hour'])['CI_HOUR']
train['mean_enc_ARI_CO_weekday_hour'] = grouped.transform('mean')
train['std_enc_ARI_CO_weekday_hour'] = grouped.transform('std')

# 해당 encoding을 test 데이터에도 적용
mean_enc_map = train.groupby(['ARI_CO', 'weekday', 'rounded_hour'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_CO', 'weekday', 'rounded_hour'])['CI_HOUR'].std().to_dict()

test['mean_enc_ARI_CO_weekday_hour'] = test.set_index(['ARI_CO', 'weekday', 'rounded_hour']).index.map(mean_enc_map).values
test['std_enc_ARI_CO_weekday_hour'] = test.set_index(['ARI_CO', 'weekday', 'rounded_hour']).index.map(std_enc_map).values
'''
'''
# 해당 encoding을 test 데이터에도 적용
grouped = train.groupby(['ARI_CO', 'weekday'])['CI_HOUR']
train['mean_enc_ARI_CO_weekday'] = grouped.transform('mean')
train['std_enc_ARI_CO_weekday'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_CO', 'weekday'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_CO', 'weekday'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_CO_weekday'] = test.set_index(['ARI_CO', 'weekday']).index.map(mean_enc_map).values
test['std_enc_ARI_CO_weekday'] = test.set_index(['ARI_CO', 'weekday']).index.map(std_enc_map).values
train['std_enc_ARI_CO_weekday'].fillna(0, inplace=True)
test['std_enc_ARI_CO_weekday'].fillna(0, inplace=True)


# 해당 encoding을 test 데이터에도 적용
ouped = train.groupby(['ARI_CO', 'month'])['CI_HOUR']
train['mean_enc_ARI_CO_month'] = grouped.transform('mean')
train['std_enc_ARI_CO_month'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_CO', 'month'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_CO', 'month'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_CO_month'] = test.set_index(['ARI_CO', 'month']).index.map(mean_enc_map).values
test['std_enc_ARI_CO_month'] = test.set_index(['ARI_CO', 'month']).index.map(std_enc_map).values
train['std_enc_ARI_CO_month'].fillna(0, inplace=True)
test['std_enc_ARI_CO_month'].fillna(0, inplace=True)


# 해당 encoding을 test 데이터에도 적용
ouped = train.groupby(['ARI_CO', 'month', 'weekday'])['CI_HOUR']
train['mean_enc_ARI_CO_month_weekday'] = grouped.transform('mean')
train['std_enc_ARI_CO_month_weekday'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_CO', 'month', 'weekday'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_CO', 'month', 'weekday'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_CO_month_weekday'] = test.set_index(['ARI_CO', 'month', 'weekday']).index.map(mean_enc_map).values
test['std_enc_ARI_CO_month_weekday'] = test.set_index(['ARI_CO', 'month', 'weekday']).index.map(std_enc_map).values
train['std_enc_ARI_CO_month_weekday'].fillna(0, inplace=True)
test['std_enc_ARI_CO_month_weekday'].fillna(0, inplace=True)


# 해당 encoding을 test 데이터에도 적용
ouped = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR']
train['mean_enc_ARI_PO_month_weekday'] = grouped.transform('mean')
train['std_enc_ARI_PO_month_weekday'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(mean_enc_map).values
test['std_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(std_enc_map).values
train['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)
test['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)



# 해당 encoding을 test 데이터에도 적용
ouped = train.groupby(['ARI_PO', 'weekday'])['CI_HOUR']
train['mean_enc_ARI_PO_month_weekday'] = grouped.transform('mean')
train['std_enc_ARI_PO_month_weekday'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(mean_enc_map).values
test['std_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(std_enc_map).values
train['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)
test['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)


# 해당 encoding을 test 데이터에도 적용
ouped = train.groupby(['ARI_PO', 'month'])['CI_HOUR']
train['mean_enc_ARI_PO_month_weekday'] = grouped.transform('mean')
train['std_enc_ARI_PO_month_weekday'] = grouped.transform('std')
mean_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].mean().to_dict()
std_enc_map = train.groupby(['ARI_PO', 'month', 'weekday'])['CI_HOUR'].std().to_dict()
test['mean_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(mean_enc_map).values
test['std_enc_ARI_PO_month_weekday'] = test.set_index(['ARI_PO', 'month', 'weekday']).index.map(std_enc_map).values
train['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)
test['std_enc_ARI_PO_month_weekday'].fillna(0, inplace=True)

'''

'''
train['std_enc_ARI_CO_weekday_hour'].fillna(0, inplace=True)
test['std_enc_ARI_CO_weekday_hour'].fillna(0, inplace=True)

# train 데이터에서 ARI_CO와 weekday 조합에 따른 mean 값을 계산
mean_enc_map_ari_co_weekday = train.groupby(['ARI_CO', 'weekday'])['CI_HOUR'].mean().to_dict()

# test 데이터에서 결측값을 해당 mean 값으로 채워주기
test['mean_enc_ARI_CO_weekday_hour'].fillna(test.set_index(['ARI_CO', 'weekday']).index.map(mean_enc_map_ari_co_weekday).values, inplace=True)
# train 데이터에서 ARI_CO 조합에 따른 mean 값을 계산
mean_enc_map_ari_co = train.groupby(['ARI_CO'])['CI_HOUR'].mean().to_dict()

# test 데이터에서 결측값을 해당 mean 값으로 채워주기
fill_values_ari_co = pd.Series(test['ARI_CO'].map(mean_enc_map_ari_co), index=test.index)
test['mean_enc_ARI_CO_weekday_hour'].fillna(fill_values_ari_co, inplace=True)

'''
# 'DUBAI', 'BRENT', 'WTI',
from sklearn.preprocessing import MinMaxScaler

features_to_scale = ['DUBAI', 'WTI', 'BRENT']
scaler = MinMaxScaler(feature_range=(0, 100))
train[features_to_scale] = scaler.fit_transform(train[features_to_scale])
test[features_to_scale] = scaler.transform(test[features_to_scale])

train['DUBAI_BRENT'] = train['DUBAI'] + train['BRENT']
train['DUBAI_WTI'] = train['DUBAI'] + train['WTI']
train['BRENT_WTI'] = train['BRENT'] + train['WTI']
train['DUBAI_BRENT_WTI'] = train['DUBAI'] + train['BRENT'] + train['WTI']

test['DUBAI_BRENT'] = test['DUBAI'] + test['BRENT']
test['DUBAI_WTI'] = test['DUBAI'] + test['WTI']
test['BRENT_WTI'] = test['BRENT'] + test['WTI']
test['DUBAI_BRENT_WTI'] = test['DUBAI'] + test['BRENT'] + test['WTI']

test

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,mean_enc_day,std_enc_day,mean_enc_weekday,std_enc_weekday,mean_enc_rounded_hour,std_enc_rounded_hour,DUBAI_BRENT,DUBAI_WTI,BRENT_WTI,DUBAI_BRENT_WTI
0,JP,HYG5,Tanker,9.508139,R721438,20.0,9,5510,10.0,10.0,...,99.269587,204.832111,160.614133,236.913654,97.697088,201.667547,144.796982,152.612162,149.849234,223.629189
1,SG,GIW5,Container,24.186684,G185231,40.0,14,109000,20.0,10.0,...,99.269587,204.832111,108.406656,207.392917,103.780640,207.373622,152.525524,159.688869,164.199700,238.207046
2,TW,EKP8,Cargo,0.000000,P862781,20.0,14,13700,10.0,10.0,...,100.907119,203.251035,90.842366,202.567329,96.976710,199.178563,121.801723,133.968688,133.013721,194.392066
3,JP,HYG5,Bulk,0.000000,B415437,10.0,18,2840,10.0,10.0,...,100.314370,196.641885,87.803141,195.561214,103.974608,210.640285,42.834313,69.968265,68.496124,90.649351
4,SG,GIW5,Container,19.834186,J412562,30.0,23,30700,20.0,10.0,...,103.141252,214.851043,87.803141,195.561214,101.085320,207.179760,123.017593,134.529014,134.475066,196.010837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244984,RU,ZME5,Bulk,1.625700,R355837,10.0,35,3120,10.0,0.0,...,99.227510,205.385334,85.508007,188.359476,101.961888,206.196032,105.576973,118.005274,117.057845,170.320046
244985,RU,ZME5,Bulk,30.642841,P438645,30.0,17,46400,20.0,10.0,...,100.799906,207.421689,160.614133,236.913654,109.006623,228.232254,88.379317,104.486010,103.028000,147.946664
244986,CN,RGT8,Cargo,0.000000,R763233,20.0,12,5280,10.0,10.0,...,105.413494,214.522677,98.936224,216.337699,109.034067,222.568626,63.478400,87.059985,85.486785,118.012585
244987,VN,ONW1,Container,0.000000,F587712,20.0,27,7020,10.0,10.0,...,108.108772,213.026799,90.842366,202.567329,105.699271,212.570578,143.019526,151.472391,152.162023,223.326970


In [40]:
train['DUBAI_BRENT'] = train['DUBAI'] * train['BRENT']
train['DUBAI_WTI'] = train['DUBAI'] * train['WTI']
train['BRENT_WTI'] = train['BRENT'] * train['WTI']
train['DUBAI_BRENT_WTI'] = train['DUBAI'] * train['BRENT'] * train['WTI']

test['DUBAI_BRENT'] = test['DUBAI'] * test['BRENT']
test['DUBAI_WTI'] = test['DUBAI'] * test['WTI']
test['BRENT_WTI'] = test['BRENT'] * test['WTI']
test['DUBAI_BRENT_WTI'] = test['DUBAI'] * test['BRENT'] * test['WTI']

In [41]:
train[train['WTI'] < 0].iloc[:,15:]

,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,...,mean_enc_day,std_enc_day,mean_enc_weekday,std_enc_weekday,mean_enc_rounded_hour,std_enc_rounded_hour,DUBAI_BRENT,DUBAI_WTI,BRENT_WTI,DUBAI_BRENT_WTI


In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244989 entries, 0 to 244988
Data columns (total 55 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ARI_CO                 244989 non-null  object 
 1   ARI_PO                 244989 non-null  object 
 2   SHIP_TYPE_CATEGORY     244989 non-null  object 
 3   DIST                   244989 non-null  float64
 4   ID                     244989 non-null  object 
 5   BREADTH                244989 non-null  float64
 6   BUILT                  244989 non-null  int64  
 7   DEADWEIGHT             244989 non-null  int64  
 8   DEPTH                  244989 non-null  float64
 9   DRAUGHT                244989 non-null  float64
 10  GT                     244989 non-null  int64  
 11  LENGTH                 244989 non-null  float64
 12  SHIPMANAGER            244989 non-null  object 
 13  FLAG                   244989 non-null  object 
 14  U_WIND                 143062 non-nu

# V3

In [43]:
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt

# 각 국가의 중심 위도와 경도
countries_coordinates = {
    'CN': (35.8617, 104.1954),
    'JP': (36.2048, 138.2529),
    'RU': (61.5240, 105.3188),
    'AU': (-25.2744, 133.7751),
    'SG': (1.3521, 103.8198),
    'ZA': (-30.5595, 22.9375),
    'KR': (35.9078, 127.7669),
    'TW': (23.6978, 120.9605),
    'TT': (10.6918, -61.2225),
    'ID': (-0.7893, 113.9213),
    'BR': (-14.2350, -51.9253),
    'QA': (25.3548, 51.1839),
    'LV': (56.8796, 24.6032),
    'MZ': (-18.6657, 35.5296),
    'US': (37.0902, -95.7129),
    'IN': (20.5937, 78.9629),
    'UA': (48.3794, 31.1656),
    'CA': (56.1304, -106.3468),
    'MY': (4.2105, 101.9758),
    'PE': (-9.1900, -75.0152),
    'VN': (14.0583, 108.2772),
    'FI': (61.9241, 25.7482),
    'CL': (-35.6751, -71.5430),
    'VE': (6.4238, -66.5897),
    'PH': (12.8797, 121.7740)
}

# 데이터 프레임 생성
coordinates_df = pd.DataFrame(countries_coordinates).T
coordinates_df.columns = ['Latitude', 'Longitude']

# 최적의 클러스터 수를 찾기 위한 Elbow Method
distortions = []
K = range(1, 15)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(coordinates_df)
    distortions.append(kmeanModel.inertia_)

optimal_clusters = distortions.index(min(distortions[2:5])) + 2  # 3 또는 4 클러스터를 선택

# KMeans 클러스터링 수행
kmeanModel = KMeans(n_clusters=optimal_clusters)
kmeanModel.fit(coordinates_df)

# 각 국가에 할당된 클러스터 레이블 출력
countries_labels = dict(zip(countries_coordinates.keys(), kmeanModel.labels_))
# train 및 test 데이터프레임에 ARI_CO 기반으로 클러스터링 레이블 추가
train['country_cluster'] = train['ARI_CO'].map(countries_labels)
test['country_cluster'] = test['ARI_CO'].map(countries_labels)

train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,std_enc_day,mean_enc_weekday,std_enc_weekday,mean_enc_rounded_hour,std_enc_rounded_hour,DUBAI_BRENT,DUBAI_WTI,BRENT_WTI,DUBAI_BRENT_WTI,country_cluster
0,CN,EKP8,Bulk,30.736578,Z517571,30.0,28,73100,20.0,10.0,...,203.933754,98.936224,216.337699,99.381610,201.303551,546.497995,1213.797997,1068.430966,26622.002999,5
1,CN,EUC8,Container,63.220425,U467618,30.0,15,37900,20.0,10.0,...,212.879594,87.803141,195.561214,101.961888,206.196032,1965.969685,2839.214807,2501.630073,118167.780538,5
2,CN,NGG6,Container,90.427421,V378315,50.0,7,115000,20.0,10.0,...,211.720962,160.614133,236.913654,105.699271,212.570578,1961.489783,2654.618480,2538.986826,114980.529854,5
3,RU,NNC2,Container,8.813725,D215135,30.0,10,27600,10.0,10.0,...,212.673518,160.614133,236.913654,101.197570,203.194964,4602.285060,5243.917696,5331.314374,358700.380279,5
4,CN,NGG6,Container,81.435335,Z156413,30.0,22,18100,10.0,10.0,...,207.421689,87.803141,195.561214,101.036616,204.252692,766.896608,1459.800265,1408.983864,39716.240979,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220050,IN,UJM2,Bulk,30.199074,Y242521,30.0,2,63500,20.0,10.0,...,211.720962,90.842366,202.567329,101.257372,205.632412,8026.135630,8083.967025,8758.887331,753858.756571,0
220051,CN,QQW1,Bulk,55.408765,D236761,30.0,16,26500,10.0,10.0,...,209.272356,98.936224,216.337699,101.036616,204.252692,7242.564844,7573.156942,7960.526239,660777.982621,5
220052,CN,YRT6,Bulk,59.018184,J661243,40.0,13,93200,20.0,10.0,...,217.267351,160.614133,236.913654,103.943272,210.636630,1647.472950,2414.901262,2283.132825,95306.917741,5
220053,SG,GIW5,Container,1.768630,Q635545,30.0,6,25000,20.0,10.0,...,227.188738,98.936224,216.337699,97.697088,201.667547,5407.236199,6090.300616,6071.319832,447145.218708,0


In [44]:

features_to_convert = ['SHIP_TYPE_CATEGORY', 'BREADTH', 'DEPTH', 'DRAUGHT', 'country_cluster']

for col in features_to_convert:
    train[col] = train[col].astype('object')
    if col in test.columns:  # test 데이터셋에도 해당 컬럼이 있으면 같이 변환
        test[col] = test[col].astype('object')


In [45]:
# 지정된 구역에 따라 PORT_SIZE를 나눔
bins = [0, 0.0007, 0.0015, 0.002, float('inf')]
labels = ['Zone_1', 'Zone_2', 'Zone_3', 'Zone_4']

#train['PORT_SIZE_Zone'] = pd.cut(train['PORT_SIZE'], bins=bins, labels=labels, include_lowest=True)
#test['PORT_SIZE_Zone'] = pd.cut(test['PORT_SIZE'], bins=bins, labels=labels, include_lowest=True)
train['PORT_SIZE_Zone'] = pd.cut(train['PORT_SIZE'], bins=bins, labels=labels, include_lowest=True).astype('object')
test['PORT_SIZE_Zone'] = pd.cut(test['PORT_SIZE'], bins=bins, labels=labels, include_lowest=True).astype('object')

#train.drop(['PORT_SIZE'],axis=1,inplace=True)
#test.drop(['PORT_SIZE'],axis=1,inplace=True)
test

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,mean_enc_weekday,std_enc_weekday,mean_enc_rounded_hour,std_enc_rounded_hour,DUBAI_BRENT,DUBAI_WTI,BRENT_WTI,DUBAI_BRENT_WTI,country_cluster,PORT_SIZE_Zone
0,JP,HYG5,Tanker,9.508139,R721438,20.0,9,5510,10.0,10.0,...,160.614133,236.913654,97.697088,201.667547,5239.633033,5816.236668,5598.429005,413051.837352,5,Zone_1
1,SG,GIW5,Container,24.186684,G185231,40.0,14,109000,20.0,10.0,...,108.406656,207.392917,103.780640,207.373622,5810.921980,6341.062114,6727.556992,497888.641441,0,Zone_4
2,TW,EKP8,Cargo,0.000000,P862781,20.0,14,13700,10.0,10.0,...,90.842366,202.567329,96.976710,199.178563,3708.686947,4455.475123,4386.153708,269214.856738,5,Zone_1
3,JP,HYG5,Bulk,0.000000,B415437,10.0,18,2840,10.0,10.0,...,87.803141,195.561214,103.974608,210.640285,458.152798,1059.257390,988.866905,21906.593241,5,Zone_1
4,SG,GIW5,Container,19.834186,J412562,30.0,23,30700,20.0,10.0,...,87.803141,195.561214,101.085320,207.179760,3783.331312,4491.695491,4487.757641,276157.624278,0,Zone_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244984,RU,ZME5,Bulk,1.625700,R355837,10.0,35,3120,10.0,0.0,...,85.508007,188.359476,101.961888,206.196032,2786.399884,3448.358543,3387.019126,180400.091658,5,Zone_1
244985,RU,ZME5,Bulk,30.642841,P438645,30.0,17,46400,20.0,10.0,...,160.614133,236.913654,109.006623,228.232254,1952.194475,2675.685598,2588.835802,116287.044583,5,Zone_1
244986,CN,RGT8,Cargo,0.000000,R763233,20.0,12,5280,10.0,10.0,...,98.936224,216.337699,109.034067,222.568626,1006.758088,1773.767995,1687.974807,54902.731390,5,Zone_1
244987,VN,ONW1,Container,0.000000,F587712,20.0,27,7020,10.0,10.0,...,90.842366,202.567329,105.699271,212.570578,5113.527276,5715.074994,5770.457600,410654.307217,0,Zone_1


In [46]:
train.drop(['U_WIND', 'V_WIND', 'AIR_TEMPERATURE','BN'],axis=1,inplace=True)
test.drop(['U_WIND', 'V_WIND', 'AIR_TEMPERATURE','BN'],axis=1,inplace=True)

In [47]:
# 필요없어보이는거
train.drop(['SHIPMANAGER'],axis=1,inplace=True)
test.drop(['SHIPMANAGER'],axis=1,inplace=True)
train.drop(['BUILT'],axis=1,inplace=True)
test.drop(['BUILT'],axis=1,inplace=True)


In [48]:
# 가정: 'hour'라는 이름의 시간 피처가 있다고 가정합니다.
train['binned_hour'] = train['ATA_LT'] // 3
test['binned_hour'] = test['ATA_LT'] // 3

In [49]:
train['PO_Y_M_D'] = train['ARI_PO'].astype(str) + "_" + train['weekday'].astype(str)
test['PO_Y_M_D'] = test['ARI_PO'].astype(str) + "_" + test['weekday'].astype(str)
unique_in_test_only = set(test['PO_Y_M_D']) - set(train['PO_Y_M_D'])
test_samples_unique = test[test['PO_Y_M_D'].isin(unique_in_test_only)]
test_samples_unique

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,DEADWEIGHT,DEPTH,DRAUGHT,GT,...,mean_enc_rounded_hour,std_enc_rounded_hour,DUBAI_BRENT,DUBAI_WTI,BRENT_WTI,DUBAI_BRENT_WTI,country_cluster,PORT_SIZE_Zone,binned_hour,PO_Y_M_D
30004,CN,RGT8,Bulk,28.265609,G463522,10.0,3680,10.0,10.0,2000,...,109.034067,222.568626,998.883446,1780.818557,1602.067877,53383.580665,5,Zone_1,6,RGT8_6
56395,CN,RGT8,Bulk,29.401089,I325466,20.0,8000,10.0,10.0,5900,...,101.961888,206.196032,747.889345,1476.909227,1414.639566,39529.240980,5,Zone_1,3,RGT8_4
78042,CN,RGT8,Bulk,31.162879,E215754,10.0,2840,10.0,10.0,2000,...,101.085320,207.179760,652.324338,1292.212773,1267.197895,32682.933283,5,Zone_1,2,RGT8_6
91897,ID,DIN2,Tanker,0.000000,F284788,20.0,5360,10.0,10.0,3270,...,105.699271,212.570578,7063.937083,7535.218577,7825.564354,645400.313638,0,Zone_1,4,DIN2_4
110874,KR,RKA2,Tanker,3.505856,L262821,10.0,2980,10.0,10.0,2990,...,96.976710,199.178563,4724.924598,5231.493423,5468.940262,367673.113497,5,Zone_1,3,RKA2_5
152470,ID,DIN2,Tanker,0.000000,P578812,20.0,1210,10.0,0.0,2420,...,101.197570,203.194964,7967.818830,8365.230941,8963.134876,772927.320910,0,Zone_1,6,DIN2_4
213725,CN,RGT8,Bulk,0.000000,A165342,10.0,2600,10.0,0.0,1980,...,109.034067,222.568626,1099.800765,1829.868911,1672.726558,58020.233770,5,Zone_1,6,RGT8_4
244988,VN,ONW1,Container,3.247493,F587712,20.0,7020,10.0,10.0,4910,...,101.036616,204.252692,3699.610908,4408.022364,4475.548898,270161.260229,0,Zone_1,7,ONW1_5


In [50]:
#train.drop(['WTI','DUBAI','BRENT'],axis=1,inplace=True)
#test.drop(['WTI','DUBAI','BRENT'],axis=1,inplace=True)

In [51]:
train.drop(['ATA_datetime'],axis=1,inplace=True)
#test.drop(['ATA_datetime'],axis=1,inplace=True)

In [52]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220055 entries, 0 to 220054
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ARI_CO                       220055 non-null  object 
 1   ARI_PO                       220055 non-null  object 
 2   SHIP_TYPE_CATEGORY           220055 non-null  object 
 3   DIST                         220055 non-null  float64
 4   ID                           220055 non-null  object 
 5   BREADTH                      220055 non-null  object 
 6   DEADWEIGHT                   220055 non-null  int64  
 7   DEPTH                        220055 non-null  object 
 8   DRAUGHT                      220055 non-null  object 
 9   GT                           220055 non-null  int64  
 10  LENGTH                       220055 non-null  float64
 11  FLAG                         220055 non-null  object 
 12  ATA_LT                       220055 non-null  int64  
 13 

In [21]:
train_x = train.drop(['CI_HOUR'],axis=1)
train_y = train[['CI_HOUR']]

# AG

## 5fold

In [53]:

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
'''
# ARI_CO + weekday 결합 열 생성
train['fold_column'] = train['ARI_CO'].astype(str) + "_" + train['weekday'].astype(str)

# Stratified 5-fold 생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=313)

# 각 폴드마다의 예측값과 MAE를 저장할 리스트
predictions = []
mae_scores = []

# StratifiedKFold는 레이블 분포를 기반으로 하기 때문에, 너무 적은 빈도의 조합은 오류를 발생시킬 수 있습니다.
min_samples = train['fold_column'].value_counts().min()
filtered_train = train[train['fold_column'].map(train['fold_column'].value_counts()) > min_samples]

for train_idx, val_idx in skf.split(filtered_train, filtered_train['fold_column']):
    train_fold = filtered_train.iloc[train_idx].drop(columns=['fold_column'])
    val_fold = filtered_train.iloc[val_idx].drop(columns=['fold_column'])
    train_data = TabularDataset(train_fold)
    val_data = TabularDataset(val_fold)
    test_data = TabularDataset(test)
    
    # 각 폴드마다 모델 학습
    predictor = TabularPredictor(label='CI_HOUR', eval_metric='mean_absolute_error').fit(
        train_data, 
        presets='medium_quality',
        ag_args_fit={'num_gpus': 0},
        included_model_types=['CAT']
    )
    
    # validation set에 대한 예측값 계산
    val_predictions = predictor.predict(val_data.drop(columns='CI_HOUR'))
    
    # MAE 계산 후 저장
    mae = mean_absolute_error(val_data['CI_HOUR'], val_predictions)
    mae_scores.append(mae)
    
    # test set에 대한 예측값 저장
    predictions.append(predictor.predict(test_data))

# 예측값의 평균 계산
final_predictions = np.mean(predictions, axis=0)
y_pred = pd.DataFrame(final_predictions, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
mae_scores, np.mean(mae_scores)
'''

'\n# ARI_CO + weekday 결합 열 생성\ntrain[\'fold_column\'] = train[\'ARI_CO\'].astype(str) + "_" + train[\'weekday\'].astype(str)\n\n# Stratified 5-fold 생성\nskf = StratifiedKFold(n_splits=5, shuffle=True, random_state=313)\n\n# 각 폴드마다의 예측값과 MAE를 저장할 리스트\npredictions = []\nmae_scores = []\n\n# StratifiedKFold는 레이블 분포를 기반으로 하기 때문에, 너무 적은 빈도의 조합은 오류를 발생시킬 수 있습니다.\nmin_samples = train[\'fold_column\'].value_counts().min()\nfiltered_train = train[train[\'fold_column\'].map(train[\'fold_column\'].value_counts()) > min_samples]\n\nfor train_idx, val_idx in skf.split(filtered_train, filtered_train[\'fold_column\']):\n    train_fold = filtered_train.iloc[train_idx].drop(columns=[\'fold_column\'])\n    val_fold = filtered_train.iloc[val_idx].drop(columns=[\'fold_column\'])\n    train_data = TabularDataset(train_fold)\n    val_data = TabularDataset(val_fold)\n    test_data = TabularDataset(test)\n    \n    # 각 폴드마다 모델 학습\n    predictor = TabularPredictor(label=\'CI_HOUR\', eval_metric=\'mean_absolute_

In [54]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)


predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['CAT'])
#predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0})

predictor.leaderboard(train, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231010_083649\"
Presets specified: ['medium_quality']


	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231010_083649\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   14.29 GB / 511.09 GB (2.8%)
Train Data Rows:    220055
Train Data Columns: 61
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.002777778, 103.30825, 210.47157)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass'

KeyError: "8 required columns are missing from the provided dataset to transform using AutoMLPipelineFeatureGenerator. 8 missing columns: ['port_daily_arrival_rate', 'port_daily_service_rate', 'port_dayofweek_arrival_rate', 'port_dayofweek_service_rate', 'P_n', 'P_busy', 'P_n_dayofweek', 'P_busy_dayofweek'] | 53 available columns: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ID', 'BREADTH', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 'FLAG', 'ATA_LT', 'DUBAI', 'BRENT', 'WTI', 'BDI_ADJ', 'PORT_SIZE', 'year', 'month', 'day', 'weekday', 'rounded_hour', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'rounded_hour_sin', 'rounded_hour_cos', 'mean_enc_ARI_CO', 'std_enc_ARI_CO', 'mean_enc_ARI_PO', 'std_enc_ARI_PO', 'mean_enc_year', 'std_enc_year', 'mean_enc_month', 'std_enc_month', 'mean_enc_day', 'std_enc_day', 'mean_enc_weekday', 'std_enc_weekday', 'mean_enc_rounded_hour', 'std_enc_rounded_hour', 'DUBAI_BRENT', 'DUBAI_WTI', 'BRENT_WTI', 'DUBAI_BREN

# Pycaret

In [29]:
from pycaret.regression import *
import category_encoders

reg = setup(data=train, target='CI_HOUR', use_gpu=True, session_id=7777, encoding_method=category_encoders.target_encoder.TargetEncoder(smoothing=10))
reg.pipeline

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

,Description,Value
0,Session id,7777
1,Target,CI_HOUR
2,Target type,Regression
3,Original data shape,"(220055, 54)"
4,Transformed data shape,"(220055, 99)"
5,Transformed train set shape,"(154038, 99)"
6,Transformed test set shape,"(66017, 99)"
7,Numeric features,42
8,Categorical features,11
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

Pipeline(memory=FastMemory(location=C:\Users\ineeji\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['DIST', 'DEADWEIGHT', 'GT',
                                             'LENGTH', 'ATA_LT', 'DUBAI',
                                             'BRENT', 'WTI', 'BDI_ADJ',
                                             'PORT_SIZE', 'year', 'month',
                                             'day', 'weekday', 'rounded_hour',
                                             'month_sin', 'month_cos',
                                             'day_sin', 'day_cos',
                                             'weekday_sin', 'weekday_cos',
                                             'rounded_h...
                                             'country_cluster',
                                             'PORT_SIZE_Zone'],
                                    transformer=OneHotEncoder(cols=['ARI_CO',
                                                                    'SHIP_TYPE_CATEGORY',
                                                                    'BREADTH',
                                                                    'DEPTH',
                                                                    'DRAUGHT',
                                                                    'country_cluster',
                                                                    'PORT_SIZE_Zone'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['ARI_PO', 'ID', 'FLAG',
                                             'PO_Y_M_D'],
                                    transformer=TargetEncoder(cols=['ARI_PO',
                                                                    'ID',
                                                                    'FLAG',
                                                                    'PO_Y_M_D'])))])

In [25]:
best = compare_models(n_select=1, sort='MAE')
best

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,77.2377,26911.8789,164.0356,0.3946,1.3282,7.1303,3.8350
catboost,CatBoost Regressor,78.0802,25976.1109,161.1658,0.4156,1.3420,7.0373,8.2230
huber,Huber Regressor,84.0494,45752.0485,213.8861,-0.0289,1.3456,4.1945,9.3980
lightgbm,Light Gradient Boosting Machine,91.3349,37766.2811,194.3288,0.1505,1.4488,8.2974,4.2230
et,Extra Trees Regressor,95.0498,40826.4559,202.0445,0.0816,1.4442,8.7758,52.8620
rf,Random Forest Regressor,96.0423,41073.9277,202.6578,0.0761,1.4503,9.5445,99.9780
gbr,Gradient Boosting Regressor,96.1261,40646.6879,201.6049,0.0857,1.4891,8.9363,79.1650
dummy,Dummy Regressor,110.1183,44468.0480,210.8644,-0.0000,1.7870,12.2057,2.5600
ridge,Ridge Regression,112.1675,42056.7184,205.0715,0.0541,1.7062,11.9721,2.4260
lr,Linear Regression,112.1713,42055.9042,205.0696,0.0541,1.7062,11.9704,3.4840


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=313, ...)

In [32]:
cat = create_model("catboost")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,78.5710,27194.3716,164.9072,0.4222,1.3286,6.2403
1,79.4191,27959.4764,167.2109,0.3765,1.3506,6.6535
2,79.5921,27532.2625,165.9285,0.4168,1.3458,7.4735
3,77.0645,25274.2824,158.9789,0.4093,1.3534,7.1385
4,78.5889,27516.9818,165.8824,0.3994,1.3355,7.4442
5,79.9769,28196.6505,167.9186,0.4079,1.3314,7.3918
6,78.0806,25307.2327,159.0825,0.4239,1.3482,6.6870
7,75.8661,24517.9706,156.5822,0.4344,1.3338,6.4500
8,76.3029,23725.4392,154.0306,0.4424,1.3377,6.4947


In [35]:
cat_t = tune_model(cat, optimize='MAE', n_iter=100) #lgbm
cat_t

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,65.9738,19343.5614,139.0811,0.5890,1.2451,5.3571
1,66.4681,19241.4675,138.7136,0.5709,1.2778,5.8650
2,65.6116,19148.8556,138.3794,0.5944,1.2603,5.8653
3,66.0880,18530.2779,136.1260,0.5669,1.2882,5.9141
4,66.2902,19988.4980,141.3807,0.5637,1.2574,6.3566
5,68.6930,20331.2303,142.5876,0.5731,1.2665,6.4792
6,65.8405,18213.8855,134.9588,0.5854,1.2697,5.8856
7,63.3068,17203.9212,131.1637,0.6031,1.2486,5.1833
8,65.2960,17623.2784,132.7527,0.5858,1.2753,5.6538


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [36]:
best_tuned = tune_model(cat_t, optimize='MAE', n_iter=100) #lgbm
best_tuned

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:03:18
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


KeyboardInterrupt: 

In [37]:
stack_lr = stack_models(cat_t, optimize='MAE', choose_better=True)
stack_lr

TypeError: 'CatBoostRegressor' object is not iterable

In [39]:
stack_finalized = finalize_model(cat_t)
stack_finalized

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['DIST', 'DEADWEIGHT', 'GT',
                                             'LENGTH', 'ATA_LT', 'DUBAI',
                                             'BRENT', 'WTI', 'BDI_ADJ',
                                             'PORT_SIZE', 'year', 'month',
                                             'day', 'weekday', 'rounded_hour',
                                             'month_sin', 'month_cos',
                                             'day_sin', 'day_cos',
                                             'weekday_sin', 'weekday_cos',
                                             'rounded_hour_sin',
                                             'rounded_hour_cos',
                                             'mean_enc_ARI...
                                                                    'SHIP_TYPE_CATEGORY',
                                                                    'BREADTH',
                                                                    'DEPTH',
                                                                    'DRAUGHT',
                                                                    'country_cluster',
                                                                    'PORT_SIZE_Zone'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['ARI_PO', 'ID', 'FLAG',
                                             'PO_Y_M_D'],
                                    transformer=TargetEncoder(cols=['ARI_PO',
                                                                    'ID',
                                                                    'FLAG',
                                                                    'PO_Y_M_D']))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x000001F8D5A571C0>)])

In [56]:
submission = pd.read_csv('(f)bestaccuracy_nofit_scaled_sample_submission.csv')
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('(f)bestaccuracy_nofit_scaled_sample_submission_2.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,90.917114
1,TEST_000001,339.572906
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,24.792130
...,...,...
244984,TEST_244984,130.493073
244985,TEST_244985,383.237549
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


In [40]:
submission['CI_HOUR'] = stack_finalized.predict(test)
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('pycat.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,98.878138
1,TEST_000001,206.534507
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,67.811210
...,...,...
244984,TEST_244984,29.505385
244985,TEST_244985,205.425372
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


# CAT

In [60]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy as np


def objective(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
            'iterations':trial.suggest_int("iterations", 300, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.1, 1),
            'depth': trial.suggest_int('depth',5, 16),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'reg_lambda': trial.suggest_uniform('reg_lambda',30,100),
            'subsample': trial.suggest_uniform('subsample',0.3,1),
            'random_strength': trial.suggest_uniform('random_strength',10,100),
            'od_wait':trial.suggest_int('od_wait', 10, 150),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,20),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 1, 100),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0., 1.0),
            'random_state' : 313,
            'verbose' : 0,
        }
    #'task_type' : 'GPU',
    #"eval_metric":'RMSE',
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val,y_val)],cat_features=cat_features,
              verbose=False)
    cat_pred = cat.predict(X_val)
    score = mean_absolute_error(y_val, cat_pred)
    return score


In [97]:
from sklearn.preprocessing import LabelEncoder
import bisect
'''
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']

# 결측치 처리
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(train_x.mean(), inplace=True)
'''
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'country_cluster', 'PORT_SIZE_Zone', 'BREADTH', 'DEPTH', 'DRAUGHT', 'PO_Y_M_D', 'FLAG']
# 수치형 변수만 대상으로 결측치 대체
numeric_cols = train_x.select_dtypes(include=[np.number]).columns

# 훈련 데이터의 평균 계산
mean_values = train_x[numeric_cols].mean()

# 결측치 대체
train_x[numeric_cols] = train_x[numeric_cols].fillna(mean_values)
test[numeric_cols] = test[numeric_cols].fillna(mean_values)

In [98]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)


In [99]:
kf = KFold(n_splits=10, shuffle=True, random_state=1008)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=313))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best RMSE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['MLM'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-08 04:38:20,348] A new study created in memory with name: no-name-d4b11179-9f09-4b01-91fe-bacc3ca82743


Optimizing hyperparameters for fold 1...


[I 2023-10-08 04:44:30,311] Trial 0 finished with value: 56.54932100528273 and parameters: {'iterations': 416, 'learning_rate': 0.5950939294198955, 'depth': 15, 'min_data_in_leaf': 19, 'reg_lambda': 96.23724709797567, 'subsample': 0.6926005800768529, 'random_strength': 88.26287372368104, 'od_wait': 34, 'leaf_estimation_iterations': 10, 'bagging_temperature': 11.278863324734598, 'colsample_bylevel': 0.43633414956249306}. Best is trial 0 with value: 56.54932100528273.
[I 2023-10-08 04:48:20,495] Trial 1 finished with value: 39.51877743888855 and parameters: {'iterations': 848, 'learning_rate': 0.6328973842342651, 'depth': 8, 'min_data_in_leaf': 26, 'reg_lambda': 34.58816504797888, 'subsample': 0.6702438670285498, 'random_strength': 15.418176184174204, 'od_wait': 117, 'leaf_estimation_iterations': 12, 'bagging_temperature': 8.240046236200172, 'colsample_bylevel': 0.3846889167241476}. Best is trial 1 with value: 39.51877743888855.
[I 2023-10-08 04:55:17,617] Trial 2 finished with value: 43

Best RMSE for fold 1: 38.956845627351825
Best hyperparameters for fold 1: {'iterations': 959, 'learning_rate': 0.15246377938647954, 'depth': 11, 'min_data_in_leaf': 22, 'reg_lambda': 60.70860649497886, 'subsample': 0.630224100896121, 'random_strength': 41.49227012066627, 'od_wait': 129, 'leaf_estimation_iterations': 9, 'bagging_temperature': 40.6193387802687, 'colsample_bylevel': 0.9641530671864936}
0:	learn: 209.6808434	total: 146ms	remaining: 2m 20s
1:	learn: 207.5929570	total: 703ms	remaining: 5m 36s
2:	learn: 206.4174889	total: 1.16s	remaining: 6m 9s
3:	learn: 205.5849703	total: 1.76s	remaining: 7m
4:	learn: 204.6263561	total: 2.21s	remaining: 7m 2s
5:	learn: 204.1411002	total: 2.75s	remaining: 7m 17s
6:	learn: 203.4873165	total: 3.3s	remaining: 7m 29s
7:	learn: 202.8260611	total: 3.79s	remaining: 7m 30s
8:	learn: 202.5731433	total: 4.13s	remaining: 7m 15s
9:	learn: 201.3134053	total: 4.7s	remaining: 7m 25s
10:	learn: 200.8060111	total: 5.25s	remaining: 7m 32s
11:	learn: 200.761453

[I 2023-10-08 06:41:36,126] A new study created in memory with name: no-name-a5bad0cc-5028-48de-8b94-64e52911ba47


...

Best RMSE for fold 9: 39.53719776650689
Best hyperparameters for fold 9: {'iterations': 932, 'learning_rate': 0.6707699152504077, 'depth': 7, 'min_data_in_leaf': 15, 'reg_lambda': 30.26423248378008, 'subsample': 0.630224100896121, 'random_strength': 36.46720207402407, 'od_wait': 80, 'leaf_estimation_iterations': 9, 'bagging_temperature': 15.96269248640496, 'colsample_bylevel': 0.6193404395539011}
0:	learn: 206.4959717	total: 223ms	remaining: 3m 27s
1:	learn: 204.2604221	total: 430ms	remaining: 3m 19s
2:	learn: 202.0552895	total: 689ms	remaining: 3m 33s
3:	learn: 201.0725972	total: 981ms	remaining: 3m 47s
4:	learn: 200.9093155	total: 1.11s	remaining: 3m 26s
5:	learn: 200.6073513	total: 1.26s	remaining: 3m 14s
6:	learn: 200.2180500	total: 1.51s	remaining: 3m 19s
7:	learn: 199.6208566	total: 1.75s	remaining: 3m 22s
8:	learn: 198.7332833	total: 1.88s	remaining: 3m 12s
9:	learn: 198.4334071	total: 2.07s	remaining: 3m 11s
10:	learn: 197.8566946	total: 2.29s	remaining: 3m 11s
11:	learn: 197.7

[I 2023-10-08 22:39:54,298] A new study created in memory with name: no-name-4c439aa4-b6b8-4d87-9e36-84bd9ac7bafe


Optimizing hyperparameters for fold 10...


[I 2023-10-08 22:51:31,774] Trial 0 finished with value: 56.59134201174269 and parameters: {'iterations': 416, 'learning_rate': 0.5950939294198955, 'depth': 15, 'min_data_in_leaf': 19, 'reg_lambda': 96.23724709797567, 'subsample': 0.6926005800768529, 'random_strength': 88.26287372368104, 'od_wait': 34, 'leaf_estimation_iterations': 10, 'bagging_temperature': 11.278863324734598, 'colsample_bylevel': 0.43633414956249306}. Best is trial 0 with value: 56.59134201174269.
[I 2023-10-08 22:55:49,995] Trial 1 finished with value: 40.88310623035718 and parameters: {'iterations': 848, 'learning_rate': 0.6328973842342651, 'depth': 8, 'min_data_in_leaf': 26, 'reg_lambda': 34.58816504797888, 'subsample': 0.6702438670285498, 'random_strength': 15.418176184174204, 'od_wait': 117, 'leaf_estimation_iterations': 12, 'bagging_temperature': 8.240046236200172, 'colsample_bylevel': 0.3846889167241476}. Best is trial 1 with value: 40.88310623035718.
[I 2023-10-08 23:03:13,757] Trial 2 finished with value: 43

Best RMSE for fold 10: 40.36773045770164
Best hyperparameters for fold 10: {'iterations': 956, 'learning_rate': 0.6650173404473864, 'depth': 8, 'min_data_in_leaf': 22, 'reg_lambda': 55.792506057232174, 'subsample': 0.6338482090954157, 'random_strength': 40.03515015533865, 'od_wait': 149, 'leaf_estimation_iterations': 1, 'bagging_temperature': 99.63801371137798, 'colsample_bylevel': 0.7185802277906428}
0:	learn: 206.6364584	total: 256ms	remaining: 4m 4s
1:	learn: 204.1231940	total: 542ms	remaining: 4m 18s
2:	learn: 202.2868470	total: 795ms	remaining: 4m 12s
3:	learn: 202.1752991	total: 1.06s	remaining: 4m 12s
4:	learn: 201.6863019	total: 1.32s	remaining: 4m 10s
5:	learn: 201.6850300	total: 1.39s	remaining: 3m 40s
6:	learn: 201.4278804	total: 1.66s	remaining: 3m 44s
7:	learn: 200.5960486	total: 1.92s	remaining: 3m 47s
8:	learn: 200.5954997	total: 1.97s	remaining: 3m 27s
9:	learn: 200.5952012	total: 2.02s	remaining: 3m 10s
10:	learn: 200.3324715	total: 2.21s	remaining: 3m 10s
11:	learn: 2

In [100]:
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('CAT_v2_1008.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,97.257813
1,TEST_000001,336.809458
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,68.130348
...,...,...
244984,TEST_244984,156.044942
244985,TEST_244985,417.542500
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


In [58]:
train

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,nBase,SpAbs_A,...,MACCS_key_158,MACCS_key_159,MACCS_key_160,MACCS_key_161,MACCS_key_162,MACCS_key_163,MACCS_key_164,MACCS_key_165,MLM,HLM
0,3.259,400.495,5,2,8,3.259,117.37,0,0,35.689316,...,1,1,1,1,1,1,1,1,26.010,50.680
1,2.169,301.407,2,1,2,2.172,73.47,0,0,26.575899,...,1,0,1,1,1,1,1,1,29.270,50.590
2,1.593,297.358,5,0,3,1.585,62.45,2,1,29.802128,...,1,0,1,1,1,1,0,1,5.586,80.892
3,4.771,494.652,6,0,5,3.475,92.60,0,1,45.884166,...,1,1,1,1,1,1,1,1,5.710,2.000
4,2.335,268.310,3,0,1,2.337,42.43,0,0,26.308663,...,1,1,1,1,1,1,1,1,93.270,99.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,3.409,396.195,3,1,5,3.409,64.74,0,0,30.902711,...,1,0,1,1,1,0,1,1,1.556,3.079
3467,1.912,359.381,4,1,3,1.844,77.37,0,0,35.887372,...,1,1,1,1,1,1,1,1,35.560,47.630
3468,1.941,261.320,3,1,6,2.124,70.14,0,0,23.546531,...,1,1,1,1,1,1,1,1,56.150,1.790
3469,0.989,284.696,5,1,5,0.989,91.51,0,0,23.936088,...,1,1,0,1,1,1,1,1,0.030,2.770
